In [1]:
import numpy as np
import matplotlib.pyplot as plt
import laspy
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.neighbors import NearestNeighbors
from imblearn.over_sampling import SMOTE
import seaborn as sns
from statistics import fmean

In [3]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
tf.config.threading.set_inter_op_parallelism_threads(12)
tf.config.threading.set_intra_op_parallelism_threads(12)

# Import point cloud data

In [5]:
def compute_variance_for_point(i, neighbors, xyz):
    indices = neighbors.radius_neighbors(xyz[i:i+1], return_distance=False)[0]
    if len(indices) > 1:
        return np.var(xyz[indices, 2])
    return 0
    
def load_las_features_labels(file_path, max_samples_per_class):
    las = laspy.read(file_path)
    
    xyz = np.vstack((las.X * las.header.scale[0] + las.header.offset[0],
                     las.Y * las.header.scale[1] + las.header.offset[1],
                     las.Z * las.header.scale[2] + las.header.offset[2])).T

    intensity = las.intensity
    classification = las.classification

    mask = ~np.isin(classification, [0, 7, 9])
    xyz = xyz[mask]
    intensity = intensity[mask]
    classification = classification[mask]
    
    
    
    if hasattr(las, 'red') and hasattr(las, 'green') and hasattr(las, 'blue'):
        red = las.red[mask].astype(np.float32) / 65535
        green = las.green[mask].astype(np.float32) / 65535
        blue = las.blue[mask].astype(np.float32) / 65535
    else:
        red = np.zeros(len(xyz), dtype=np.float32)
        green = np.zeros(len(xyz), dtype=np.float32)
        blue = np.zeros(len(xyz), dtype=np.float32)

    # Undersampling
    if max_samples_per_class:
        class_counts = Counter(classification)
        selected_indices = []

        for cls, count in class_counts.items():
            cls_indices = np.where(classification == cls)[0]
            if count > max_samples_per_class:
                cls_indices = np.random.choice(cls_indices, max_samples_per_class, replace=False)
            selected_indices.extend(cls_indices)
    
        xyz = xyz[selected_indices]
        intensity = intensity[selected_indices]
        classification = classification[selected_indices]
        red = red[selected_indices]
        green = green[selected_indices]
        blue = blue[selected_indices]
    

    min_z = np.min(xyz[:, 2])
    height_above_ground = xyz[:, 2] - min_z

    #neighbors = NearestNeighbors(radius=1, algorithm='kd_tree').fit(xyz)
    #local_density = neighbors.radius_neighbors_graph(xyz).sum(axis=1).A1
    '''
    local_variance = Parallel(n_jobs=12)(
        delayed(compute_variance_for_point)(i, neighbors, xyz) for i in range(len(xyz))
    )
    local_variance = np.array(local_variance, dtype=np.float32)
    '''

    #One-hot encode labels
    le = LabelEncoder()
    le.fit(classification)
    classification_encoded = le.transform(classification)
    class_number = len(le.classes_)
    classification_categorical = to_categorical(classification_encoded, class_number)
    
    features = np.column_stack((intensity, height_above_ground, red, green, blue))

    return features, classification_categorical



In [7]:
def load_clouds_features(file_paths, max_samples):
    max_samples_per_class = int(max_samples / (len(file_paths) - 1))
    
    clouds = []
    for file in file_paths:
        features, labels = load_las_features_labels(file, max_samples_per_class)
        clouds.append( (features, labels) )
    return clouds

def merge_features(clouds_features):
    features = np.vstack([cloud[0] for cloud in clouds_features])
    labels = np.vstack([cloud[1] for cloud in clouds_features])
    return features, labels

def mean_iou(y_true, y_pred, num_classes):
    cm = confusion_matrix(y_true, y_pred, labels=np.arange(num_classes))
    IoUs = np.diag(cm) / (cm.sum(axis=1) + cm.sum(axis=0) - np.diag(cm) + 1e-7)
    return np.nanmean(IoUs)

# Build the model

In [33]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input((5,)),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(5, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [37]:
def cross_eval(clouds_features):
    acc = []
    
    for i, cloud in enumerate(clouds_features):
        print(f"Konfiguracja {i}")
        eval_features, eval_labels = cloud 
        train_clouds = clouds_features[:i] + clouds_features[i+1:] 
        
        train_features, train_labels = merge_features(train_clouds)

        #smote = SMOTE(sampling_strategy='auto', random_state=7)
        #train_features, train_labels = smote.fit_resample(train_features, train_labels)

        model = create_model()
        model.fit(train_features, train_labels, epochs=7)
        loss, accuracy = model.evaluate(eval_features, eval_labels)
        acc.append(accuracy)
        
        print(f"Classification Report {i}")
        print(f'accuracy: {accuracy}')
  
    overal_accuracy = fmean(acc)
    print('')
    print('Final Raport')
    print(f'OA: {overal_accuracy}')
    #plot_confusion_matrix(all_eval_labels, all_pred_labels)

In [41]:
# Ścieżki do plików LAS
file_paths = ["LAS/N-34-50-C-d-3-3-2-2.las", "LAS/N-34-50-C-d-3-3-2-4.las", "LAS/N-34-50-C-c-4-2-2-2.las", "LAS/N-34-50-C-c-4-2-2-1.las"]

clouds = load_clouds_features(file_paths, 1000000)

cross_eval(clouds)


Konfiguracja 0
Epoch 1/7
131756/131756 ━━━━━━━━━━━━━━━━━━━━ 145s 1ms/step - accuracy: 0.5492 - loss: 1.0375
Epoch 2/7
131756/131756 ━━━━━━━━━━━━━━━━━━━━ 204s 1ms/step - accuracy: 0.6107 - loss: 0.9042
Epoch 3/7
131756/131756 ━━━━━━━━━━━━━━━━━━━━ 202s 1ms/step - accuracy: 0.6200 - loss: 0.8813
Epoch 4/7
131756/131756 ━━━━━━━━━━━━━━━━━━━━ 146s 1ms/step - accuracy: 0.6243 - loss: 0.8697
Epoch 5/7
131756/131756 ━━━━━━━━━━━━━━━━━━━━ 201s 1ms/step - accuracy: 0.6273 - loss: 0.8618
Epoch 6/7
131756/131756 ━━━━━━━━━━━━━━━━━━━━ 202s 1ms/step - accuracy: 0.6294 - loss: 0.8564
Epoch 7/7
131756/131756 ━━━━━━━━━━━━━━━━━━━━ 202s 1ms/step - accuracy: 0.6309 - loss: 0.8520
37570/37570 ━━━━━━━━━━━━━━━━━━━━ 35s 936us/step - accuracy: 0.2800 - loss: 3.5321
Classification Report 0
accuracy: 0.4880838096141815
Konfiguracja 1
Epoch 1/7
130783/130783 ━━━━━━━━━━━━━━━━━━━━ 145s 1ms/step - accuracy: 0.5632 - loss: 1.0221
Epoch 2/7
122267/130783 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.6235 - loss: 0.8837

KeyboardInterrupt: 